# モジュールインポート

In [40]:
import pandas as pd
import glob
import os
import time
import re
import datetime
from tqdm.notebook import tqdm
import dataclasses
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from abc import ABCMeta, abstractmethod
from types import MappingProxyType
from sklearn.preprocessing import LabelEncoder

In [ ]:
@dataclasses.dataclass(frozen=True)
class UrlPaths:
    DB_DOMAIN: str = 'https://db.netkeiba.com/'
    # レース結果テーブル、レース情報テーブル、払い戻しテーブルが含まれるページ
    RACE_URL: str = DB_DOMAIN + 'race/'
    # 馬の過去成績テーブルが含まれるページ
    HORSE_URL: str = DB_DOMAIN + 'horse/'
    # 血統テーブルが含まれるページ
    PED_URL: str = HORSE_URL + 'ped/'
    
    TOP_URL: str = 'https://race.netkeiba.com/top/'
    # 開催日程ページ
    CALENDAR_URL: str = TOP_URL + 'calendar.html'
    # レース一覧ページ
    RACE_LIST_URL: str = TOP_URL + 'race_list.html'
    
    # 出馬表ページ
    SHUTUBA_TABLE: str = 'https://race.netkeiba.com/race/shutuba.html'

In [64]:
@dataclasses.dataclass(frozen=True)
class LocalPaths:
    # パス
    ## プロジェクトルートの絶対パス
    BASE_PATH: str = os.path.abspath('./')
    ## dataディレクトリまでの絶対パス
    DATA_PATH: str = os.path.join(os.path.abspath('./'),'data')
    ### HTMLディレクトリのパス
    HTML_PATH: str = os.path.join(DATA_PATH, 'html')
    HTML_RACE_PATH: str = os.path.join(HTML_PATH, 'race')
    HTML_HORSE_PATH: str = os.path.join(HTML_PATH, 'horse')
    HTML_PED_PATH: str = os.path.join(HTML_PATH, 'ped')
    
    ### rawディレクトリのパス
    RAW_PATH: str = os.path.join(DATA_PATH, 'raw')
    RAW_RESULTS_PATH: str = os.path.join(RAW_PATH, 'results')
    RAW_RACE_INFO_PATH: str = os.path.join(RAW_PATH, 'race_info')
    RAW_RETURN_PATH: str = os.path.join(RAW_PATH, 'return_tables')
    RAW_HORSE_RESULTS_PATH: str = os.path.join(RAW_PATH, 'horse_results')
    RAW_PEDS_PATH: str = os.path.join(RAW_PATH, 'peds')

# 関数置き場
## 各セルをそのまま実行してもらえれば大丈夫です

In [ ]:
def get_kaisai_date(from_: str, to_: str):
    date_range = pd.date_range(start=from_, end=to_, freq="M")
    kaisai_date_list = []
    for year, month in tqdm(zip(date_range.year, date_range.month), total=len(date_range)):
        query = [
            'year=' + str(year),
            'month=' + str(month),
        ]
        url = UrlPaths.CALENDAR_URL + '?' + '&'.join(query)
        html = urlopen(url).read()
        time.sleep(1)
        soup = BeautifulSoup(html, "html.parser")
        a_list = soup.find('table', class_='Calendar_Table').find_all('a')
        for a in a_list:
            kaisai_date_list.append(re.findall('(?<=kaisai_date=)\d+', a['href'])[0])
    return kaisai_date_list

In [ ]:
def get_race_id_list(kaisai_date_list: list):
    race_id_list = []
    driver = webdriver.Chrome()
    for kaisai_date in tqdm(kaisai_date_list):
        try:
            query = [
                'kaisai_date=' + str(kaisai_date)
            ]
            url = UrlPaths.RACE_LIST_URL + '?' + '&'.join(query)
            print('scraping: {}'.format(url))
            driver.get(url)
            try:
                time.sleep(1) #取得の猶予として
                a_list = driver.find_element(By.CLASS_NAME, 'RaceList_Box').find_elements(By.TAG_NAME, 'a')
            except: #それでも取得できなかった場合
                print('waiting more 10 seconds')
                time.sleep(10)
                a_list = driver.find_element(By.CLASS_NAME, 'RaceList_Box').find_elements(By.TAG_NAME, 'a')
            for a in a_list:
                race_id = re.findall('(?<=result.html\?race_id=)\d+', a.get_attribute('href'))
                if len(race_id) > 0:
                    race_id_list.append(race_id[0])
        except Exception as e:
            print(e)
            break
    driver.close()
    return race_id_list

In [70]:
def get_html_horse(horse_id_list: list, skip: bool = True):
    """
    netkeiba.comのhorseページのhtmlをスクレイピングしてdata/html/horseに保存する関数。
    """
    html_path_list = []
    for horse_id in tqdm(horse_id_list):
        url = 'https://db.netkeiba.com/horse/' + horse_id #horse_idからurlを作る
        html = urlopen(url).read() #スクレイピング実行
        filename = 'data/html/horse/' + horse_id + '.bin'
        html_path_list.append(filename)
        if skip and os.path.isfile(filename): #skipがTrueで、かつbinファイルがすでに存在する場合は飛ばす
            print('horse_id {} skipped'.format(horse_id))
            continue
        with open(filename, 'wb') as f: #保存するファイルパスを指定
            f.write(html) #保存
        time.sleep(1) #サーバーが落ちないようにするため必須
    return html_path_list

def get_html_race(race_id_list: list, skip: bool = True):
    """
    netkeiba.comのraceページのhtmlをスクレイピングしてdata/html/raceに保存する関数。
    """
    html_path_list = []
    for race_id in tqdm(race_id_list):
        url = 'https://db.netkeiba.com/race/' + race_id #race_idからurlを作る
        html = urlopen(url).read() #スクレイピング実行
        filename = 'data/html/race/' + race_id + '.bin'
        html_path_list.append(filename)
        if skip and os.path.isfile(filename): #skipがTrueで、かつbinファイルがすでに存在する場合は飛ばす
            print('race_id {} skipped'.format(race_id))
            continue
        with open(filename, 'wb') as f: #保存するファイルパスを指定
            f.write(html) #保存
        time.sleep(1) #サーバーが落ちないようにするため必須
    return html_path_list

def get_html_ped(horse_id_list: list, skip: bool = True):
    """
    netkeiba.comのhorse/pedページのhtmlをスクレイピングしてdata/html/pedに保存する関数。
    """
    html_path_list = []
    for horse_id in tqdm(horse_id_list):
        url = 'https://db.netkeiba.com/horse/ped/' + horse_id #horse_idからurlを作る
        html = urlopen(url).read() #スクレイピング実行
        filename = 'data/html/ped/' + horse_id + '.bin'
        html_path_list.append(filename)
        if skip and os.path.isfile(filename): #skipがTrueで、かつbinファイルがすでに存在する場合は飛ばす
            print('horse_id {} skipped'.format(horse_id))
            continue
        with open(filename, 'wb') as f: #保存するファイルパスを指定
            f.write(html) #保存
        time.sleep(1) #サーバーが落ちないようにするため必須
    return html_path_list

In [67]:
def get_rawdata_results(html_path_list: list):
    """
    raceページのhtmlを受け取って、レース結果テーブルに変換する関数。
    """
    race_results = {}
    for html_path in tqdm(html_path_list):
        with open(html_path, 'rb') as f:
            try:
                html = f.read() #保存してあるbinファイルを読み込む
                df = pd.read_html(html)[0] #メインとなるレース結果テーブルデータを取得
                
                soup = BeautifulSoup(html, "html.parser") #htmlをsoupオブジェクトに変換

                #馬ID、騎手IDをスクレイピング
                horse_id_list = []
                horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/horse")}
                )
                for a in horse_a_list:
                    horse_id = re.findall(r"\d+", a["href"])
                    horse_id_list.append(horse_id[0])
                jockey_id_list = []
                jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/jockey")}
                )
                for a in jockey_a_list:
                    jockey_id = re.findall(r"\d+", a["href"])
                    jockey_id_list.append(jockey_id[0])
                df["horse_id"] = horse_id_list
                df["jockey_id"] = jockey_id_list

                #インデックスをrace_idにする
                race_id = re.findall('(?<=race/)\d+', html_path)[0]
                df.index = [race_id] * len(df)

                race_results[race_id] = df
            except Exception as e:
                print('error at {}'.format(html_path))
                print(e)
    #pd.DataFrame型にして一つのデータにまとめる
    race_results_df = pd.concat([race_results[key] for key in race_results])

    return race_results_df

def get_rawdata_info(html_path_list: list):
    """
    raceページのhtmlを受け取って、レース情報テーブルに変換する関数。
    """
    race_infos = {}
    for html_path in tqdm(html_path_list):
        with open(html_path, 'rb') as f:
            try:
                html = f.read() #保存してあるbinファイルを読み込む
                
                soup = BeautifulSoup(html, "html.parser") #htmlをsoupオブジェクトに変換

                #天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
                texts = (
                    soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                    + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
                )
                info = re.findall(r'\w+', texts)
                df = pd.DataFrame()
                for text in info:
                    if text in ["芝", "ダート"]:
                        df["race_type"] = [text]
                    if "障" in text:
                        df["race_type"] = ["障害"]
                    if "m" in text:
                        df["course_len"] = [int(re.findall(r"\d+", text)[-1])] #20211212：[0]→[-1]に修正
                    if text in ["良", "稍重", "重", "不良"]:
                        df["ground_state"] = [text]
                    if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                        df["weather"] = [text]
                    if "年" in text:
                        df["date"] = [text]
                
                #インデックスをrace_idにする
                race_id = re.findall('(?<=race/)\d+', html_path)[0]
                df.index = [race_id] * len(df)

                race_infos[race_id] = df
            except Exception as e:
                print('error at {}'.format(html_path))
                print(e)
    #pd.DataFrame型にして一つのデータにまとめる
    race_infos_df = pd.concat([race_infos[key] for key in race_infos])

    return race_infos_df

def get_rawdata_return(html_path_list: list):
    """
    raceページのhtmlを受け取って、払い戻しテーブルに変換する関数。
    """
    horse_results = {}
    for html_path in tqdm(html_path_list):
        with open(html_path, 'rb') as f:
            try: 
                html = f.read() #保存してあるbinファイルを読み込む
                
                html = html.replace(b'<br />', b'br')
                dfs = pd.read_html(html)

                #dfsの1番目に単勝〜馬連、2番目にワイド〜三連単がある
                df = pd.concat([dfs[1], dfs[2]])
                
                race_id = re.findall('(?<=race/)\d+', html_path)[0]
                df.index = [race_id] * len(df)
                horse_results[race_id] = df
            except Exception as e:
                print('error at {}'.format(html_path))
                print(e)
    #pd.DataFrame型にして一つのデータにまとめる
    horse_results_df = pd.concat([horse_results[key] for key in horse_results])
    return horse_results_df

def get_rawdata_horse_results(html_path_list: list):
    """
    horseページのhtmlを受け取って、馬の過去成績のDataFrameに変換する関数。
    """
    horse_results = {}
    for html_path in tqdm(html_path_list):
        with open(html_path, 'rb') as f:
            html = f.read() #保存してあるbinファイルを読み込む
            
            df = pd.read_html(html)[3]
            #受賞歴がある馬の場合、3番目に受賞歴テーブルが来るため、4番目のデータを取得する
            if df.columns[0]=='受賞歴':
                df = pd.read_html(html)[4]
                
            horse_id = re.findall('(?<=horse/)\d+', html_path)[0]
            
            df.index = [horse_id] * len(df)
            horse_results[horse_id] = df
            
    #pd.DataFrame型にして一つのデータにまとめる
    horse_results_df = pd.concat([horse_results[key] for key in horse_results])
    return horse_results_df

def get_rawdata_peds(html_path_list: list):
    """
    horse/pedページのhtmlを受け取って、血統のDataFrameに変換する関数。
    """
    peds = {}
    for html_path in tqdm(html_path_list):
        with open(html_path, 'rb') as f:
            html = f.read() #保存してあるbinファイルを読み込む
            
            df = pd.read_html(html)[0]

            #重複を削除して1列のSeries型データに直す
            generations = {}
            horse_id = re.findall('(?<=ped/)\d+', html_path)[0]
            for i in reversed(range(5)):
                generations[i] = df[i]
                df.drop([i], axis=1, inplace=True)
                df = df.drop_duplicates()
            ped = pd.concat([generations[i] for i in range(5)]).rename(horse_id)
            peds[horse_id] = ped.reset_index(drop=True)
    #pd.DataFrame型にして一つのデータにまとめる
    peds_df = pd.concat([peds[key] for key in peds], axis=1).T.add_prefix('peds_')
    return peds_df

# A.データ取得

## レースID取得

## ※ここから自分で書き換える箇所あり
#### （書き換え方）from_とto_の所で取得したい日時の範囲を指定する

In [47]:
#開催日取得
kaisai_date_2023 = get_kaisai_date(from_="2022-01-01", to_="2023-01-01")

  0%|          | 0/1 [00:00<?, ?it/s]

9

#### （書き換え方）(kaisai_date_年数)として年数の部分を書き換える

In [49]:
# 開催日からレースIDの取得とする
race_id_list = get_race_id_list(kaisai_date_2023)

  0%|          | 0/9 [00:00<?, ?it/s]

scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20181201
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20181202
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20181208
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20181209
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20181215
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20181216
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20181222
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20181223
scraping: https://race.netkeiba.com/top/race_list.html?kaisai_date=20181228


288

## raceデータ取得

In [51]:
#https://db.netkeiba.com/race/のhtml(binファイル)をスクレイピングして保存
html_files_race = get_html_race(race_id_list)
html_files_race[:5]

  0%|          | 0/288 [00:00<?, ?it/s]

['data/html/race/201806050101.bin',
 'data/html/race/201806050102.bin',
 'data/html/race/201806050103.bin',
 'data/html/race/201806050104.bin',
 'data/html/race/201806050105.bin']

In [54]:
results = get_rawdata_results(html_files_race) #レース結果テーブルの作成
race_info = get_rawdata_info(html_files_race) #レース情報テーブルの作成
return_table = get_rawdata_return(html_files_race) #払戻テーブルの作成

  0%|          | 0/288 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

#### （書き換え方）_年数の部分を書き換えると後で使いやすくなる

In [59]:
#保存（分かりやすいように年数を書き換える）
results.to_pickle('data/raw/results/results_2023.pickle')
race_info.to_pickle('data/raw/race_info/race_info_2023.pickle')
return_table.to_pickle('data/raw/return_tables/return_tables_2023.pickle')

## horseデータ取得

In [62]:
horse_id_list = results['horse_id'].unique()
html_files_horse = get_html_horse(horse_id_list) #htmlをスクレイピング

  0%|          | 0/3512 [00:00<?, ?it/s]

In [65]:
html_files_horse = []
for horse_id in tqdm(horse_id_list):
    file = glob.glob(os.path.join(LocalPaths.HTML_HORSE_PATH, horse_id+'*.bin'))[0]
    html_files_horse.append(file)
html_files_horse[:5]

  0%|          | 0/3512 [00:00<?, ?it/s]

['/Users/yuta_watanabe/Downloads/scrape/data/html/horse/2016103911.bin',
 '/Users/yuta_watanabe/Downloads/scrape/data/html/horse/2016102397.bin',
 '/Users/yuta_watanabe/Downloads/scrape/data/html/horse/2016104661.bin',
 '/Users/yuta_watanabe/Downloads/scrape/data/html/horse/2016103526.bin',
 '/Users/yuta_watanabe/Downloads/scrape/data/html/horse/2016104091.bin']

#### （書き換え方）同様に_年数の部分を書き換えると後で扱いやすい

In [68]:
#分かりやすいように年数を書き換える
horse_results_2023 = get_rawdata_horse_results(html_files_horse) #馬の過去成績テーブルの作成
horse_results_2023.to_pickle('data/raw/horse_results/horse_results_2023.pickle')

  0%|          | 0/3512 [00:00<?, ?it/s]

## pedデータ取得

In [72]:
html_files_peds = get_html_ped(horse_id_list) #htmlをスクレイピング

  0%|          | 0/3512 [00:00<?, ?it/s]

#### （書き換え方）同様に_年数の部分を書き換えると後で扱いやすい

In [ ]:
#分かりやすいように年数を書き換える
peds_2023 = get_rawdata_peds(html_files_peds) #血統テーブルの作成
peds_2023.to_pickle('data/raw/peds/peds_2023.pickle')

  0%|          | 0/3512 [00:00<?, ?it/s]

## 関数置き場

In [ ]:
class AbstractDataProcessor(metaclass=ABCMeta):
    def __init__(self, path_list):
        self.__raw_data = self._read_pickle(path_list)
        self.__preprocessed_data = self._preprocess()

    @abstractmethod
    def _preprocess(self):
        pass
    
    @property
    def raw_data(self):
        return self.__raw_data.copy()

    @property
    def preprocessed_data(self):
        return self.__preprocessed_data.copy()

    def _delete_duplicate(self, old, new):
        filtered_old = old[~old.index.isin(new.index)]
        return pd.concat([filtered_old, new])

    def _read_pickle(self, path_list):
        df = pd.read_pickle(path_list[0])
        for path in path_list[1:]:
            df = self._delete_duplicate(df, pd.read_pickle(path))
        return df

class ResultsProcessor(AbstractDataProcessor):
    def __init__(self, path_list):
        super().__init__(path_list)
    
    def _preprocess(self):
        df = self.raw_data.copy()
        
        df = self._preprocess_rank(df)
        
        # 性齢を性と年齢に分ける
        df["性"] = df["性齢"].map(lambda x: str(x)[0])
        df["年齢"] = df["性齢"].map(lambda x: str(x)[1:]).astype(int)

        # 馬体重を体重と体重変化に分ける
        df["体重"] = df["馬体重"].str.split("(", expand=True)[0]
        df["体重変化"] = df["馬体重"].str.split("(", expand=True)[1].str[:-1]
        
        #errors='coerce'で、"計不"など変換できない時に欠損値にする
        df['体重'] = pd.to_numeric(df['体重'], errors='coerce')
        df['体重変化'] = pd.to_numeric(df['体重変化'], errors='coerce')

        # 各列を数値型に変換
        df["単勝"] = df["単勝"].astype(float)
        df["斤量"] = df["斤量"].astype(float)
        df["枠番"] = df["枠番"].astype(int)
        df["馬番"] = df["馬番"].astype(int)
        
        #6/6出走数追加
        df['n_horses'] = df.index.map(df.index.value_counts())
        
        df = self._select_columns(df)
        
        return df
        
        
    def _preprocess_rank(self, raw):
        df = raw.copy()
        # 着順に数字以外の文字列が含まれているものを取り除く
        df['着順'] = pd.to_numeric(df['着順'], errors='coerce')
        df.dropna(subset=['着順'], inplace=True)
        df['着順'] = df['着順'].astype(int)
        df['rank'] = df['着順'].map(lambda x:1 if x<4 else 0)
        return df
    
    def _select_columns(self, raw):
        df = raw.copy()[[
            '枠番','馬番','斤量','単勝','horse_id','jockey_id','性', '年齢','体重','体重変化','n_horses', 'rank'
            ]]
        return df

class RaceInfoProcessor(AbstractDataProcessor):
    def __init__(self, path_list):
        super().__init__(path_list)
        
    def _preprocess(self):
        df = self.raw_data
        # 距離は10の位を切り捨てる
        df["course_len"] = df["course_len"].astype(float) // 100
        # 日付型に変更
        df["date"] = pd.to_datetime(df["date"], format="%Y年%m月%d日")
        # 開催場所
        df['開催'] = df.index.map(lambda x:str(x)[4:6])
        
        return df
    
class ReturnProcessor(AbstractDataProcessor):
    def __init__(self, path_list):
        super().__init__(path_list)
    
    def _preprocess(self):
        return_dict = {}
        return_dict['tansho'] = self.__tansho()
        return_dict['fukusho'] = self.__fukusho()
        return_dict['umaren'] = self.__umaren()
        return_dict['umatan'] = self.__umatan()
        return_dict['wide'] = self.__wide()
        return_dict['sanrentan'] = self.__sanrentan()
        return_dict['sanrenpuku'] = self.__sanrenpuku()        
        return return_dict
    
    def __tansho(self):
        tansho = self.raw_data[self.raw_data[0]=='単勝'][[1,2]]
        tansho.columns = ['win', 'return']
        
        for column in tansho.columns:
            tansho[column] = pd.to_numeric(tansho[column], errors='coerce')
            
        return tansho
    
    def __fukusho(self):
        fukusho = self.raw_data[self.raw_data[0]=='複勝'][[1,2]]
        wins = fukusho[1].str.split('br', expand=True)[[0,1,2]]
        
        wins.columns = ['win_0', 'win_1', 'win_2']
        returns = fukusho[2].str.split('br', expand=True)[[0,1,2]]
        returns.columns = ['return_0', 'return_1', 'return_2']
        
        df = pd.concat([wins, returns], axis=1)
        for column in df.columns:
            df[column] = df[column].str.replace(',', '')
        return df.fillna(0).astype(int)
    
    
    def __umaren(self):
        umaren = self.raw_data[self.raw_data[0]=='馬連'][[1,2]]
        wins = umaren[1].str.split('-', expand=True)[[0,1]].add_prefix('win_')
        return_ = umaren[2].rename('return')  
        df = pd.concat([wins, return_], axis=1)        
        return df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    
    
    def __umatan(self):
        umatan = self.raw_data[self.raw_data[0]=='馬単'][[1,2]]
        wins = umatan[1].str.split('→', expand=True)[[0,1]].add_prefix('win_')
        return_ = umatan[2].rename('return')  
        df = pd.concat([wins, return_], axis=1)        
        return df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    
    
    def __wide(self):
        wide = self.raw_data[self.raw_data[0]=='ワイド'][[1,2]]
        wins = wide[1].str.split('br', expand=True)[[0,1,2]]
        wins = wins.stack().str.split('-', expand=True).add_prefix('win_')
        return_ = wide[2].str.split('br', expand=True)[[0,1,2]]
        return_ = return_.stack().rename('return')
        df = pd.concat([wins, return_], axis=1)
        return df.apply(lambda x: pd.to_numeric(x.str.replace(',',''), errors='coerce'))
    
    
    def __sanrentan(self):
        rentan = self.raw_data[self.raw_data[0]=='三連単'][[1,2]]
        wins = rentan[1].str.split('→', expand=True)[[0,1,2]].add_prefix('win_')
        return_ = rentan[2].rename('return')
        df = pd.concat([wins, return_], axis=1) 
        return df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    
    
    def __sanrenpuku(self):
        renpuku = self.raw_data[self.raw_data[0]=='三連複'][[1,2]]
        wins = renpuku[1].str.split('-', expand=True)[[0,1,2]].add_prefix('win_')
        return_ = renpuku[2].rename('return')
        df = pd.concat([wins, return_], axis=1) 
        return df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    
class HorseResultsProcessor(AbstractDataProcessor):
    def __init__(self, path_list):
        super().__init__(path_list)
    
    def _preprocess(self):
        df = self.raw_data

        # 着順に数字以外の文字列が含まれているものを取り除く
        df['着順'] = pd.to_numeric(df['着順'], errors='coerce')
        df.dropna(subset=['着順'], inplace=True)
        df['着順'] = df['着順'].astype(int)

        df["date"] = pd.to_datetime(df["日付"])
        df.drop(['日付'], axis=1, inplace=True)
        
        #賞金のNaNを0で埋める
        df['賞金'].fillna(0, inplace=True)
        
        #1着の着差を0にする
        df['着差'] = df['着差'].map(lambda x: 0 if x<0 else x)
        
        #レース展開データ
        #n=1: 最初のコーナー位置, n=4: 最終コーナー位置
        def corner(x, n):
            if type(x) != str:
                return x
            elif n==4:
                return int(re.findall(r'\d+', x)[-1])
            elif n==1:
                return int(re.findall(r'\d+', x)[0])
        df['first_corner'] = df['通過'].map(lambda x: corner(x, 1))
        df['final_corner'] = df['通過'].map(lambda x: corner(x, 4))
        
        df['final_to_rank'] = df['final_corner'] - df['着順']
        df['first_to_rank'] = df['first_corner'] - df['着順']
        df['first_to_final'] = df['first_corner'] - df['final_corner']
        
        #開催場所
        df['開催'] = df['開催'].str.extract(r'(\D+)')[0].map(master.PLACE_DICT).fillna('11')
        #race_type
        df['race_type'] = df['距離'].str.extract(r'(\D+)')[0].map(master.RACE_TYPE_DICT)
        #距離は10の位を切り捨てる
        df['course_len'] = df['距離'].str.extract(r'(\d+)').astype(int) // 100
        df.drop(['距離'], axis=1, inplace=True)
        #インデックス名を与える
        df.index.name = 'horse_id'
        
        return df

class PedsProcessor(AbstractDataProcessor):
    def __init__(self, path_list):
        super().__init__(path_list)
    
    def _preprocess(self):
        df = self.raw_data
        for column in df.columns:
            df[column] = LabelEncoder().fit_transform(df[column].fillna('Na'))
        return df.astype('category')
 
@dataclasses.dataclass(frozen=True)
class Master:
    PLACE_DICT: dict = MappingProxyType({
        '札幌':'01',
        '函館':'02',
        '福島':'03',
        '新潟':'04',
        '東京':'05',
        '中山':'06',
        '中京':'07',
        '京都':'08',
        '阪神':'09',
        '小倉':'10',
        })

    RACE_TYPE_DICT: dict = MappingProxyType({
        '芝': '芝',
        'ダ': 'ダート',
        '障': '障害',
        })
    
    WEATHER_LIST: tuple = ('晴', '曇', '小雨', '雨', '小雪', '雪')
    
    GROUND_STATE_LIST: tuple = ('良', '稍重', '重', '不良')
    
    SEX_LIST: tuple = ('牡', '牝', 'セ')

In [ ]:
class FeatureEngineering:
    """
    使うテーブルを全てマージした後の処理をするクラス。
    新しい特徴量を作りたいときは、メソッド単位で追加していく。
    各メソッドは依存関係を持たないよう注意。
    """
    def __init__(self, data_merger: DataMerger):
        self.__data = data_merger.merged_data.copy()
        
    @property
    def featured_data(self):
        return self.__data
    
    def add_interval(self):
        """
        前走からの経過日数
        """
        self.__data['interval'] = (self.__data['date'] - self.__data['latest']).dt.days
        self.__data.drop('latest', axis=1, inplace=True)
        return self
    
    def dumminize_weather(self):
        """
        weatherカラムをダミー変数化する
        """
        self.__data[Cols.WEATHER] = pd.Categorical(self.__data[Cols.WEATHER], Master.WEATHER_LIST)
        self.__data = pd.get_dummies(self.__data, columns=[Cols.WEATHER])
        return self
    
    def dumminize_race_type(self):
        """
        race_typeカラムをダミー変数化する
        """
        self.__data[Cols.RACE_TYPE] = pd.Categorical(
            self.__data[Cols.RACE_TYPE], list(Master.RACE_TYPE_DICT.values())
            )
        self.__data = pd.get_dummies(self.__data, columns=[Cols.RACE_TYPE])
        return self
    
    def dumminize_ground_state(self):
        """
        ground_stateカラムをダミー変数化する
        """
        self.__data[Cols.GROUND_STATE] = pd.Categorical(
            self.__data[Cols.GROUND_STATE], Master.GROUND_STATE_LIST
            )
        self.__data = pd.get_dummies(self.__data, columns=[Cols.GROUND_STATE])
        return self
    
    def dumminize_sex(self):
        """
        sexカラムをダミー変数化する
        """
        self.__data[Cols.SEX] = pd.Categorical(self.__data[Cols.SEX], Master.SEX_LIST)
        self.__data = pd.get_dummies(self.__data, columns=[Cols.SEX])
        return self
    
    def encode_horse_id(self):
        """
        horse_idをラベルエンコーディングして、Categorical型に変換する。
        """
        csv_path = 'data/master/horse_id.csv'
        horse_master = pd.read_csv(csv_path, dtype=object)
        new_horses = self.__data[[Cols.HORSE_ID]][
            ~self.__data[Cols.HORSE_ID].isin(horse_master['horse_id'])
            ].drop_duplicates(subset=['horse_id'])
        new_horses['encoded_id'] = [i+len(horse_master) for i in range(len(new_horses))]
        new_horse_master = pd.concat([horse_master, new_horses]).set_index('horse_id')['encoded_id']
        new_horse_master.to_csv(csv_path)
        self.__data[Cols.HORSE_ID] = pd.Categorical(self.__data[Cols.HORSE_ID].map(new_horse_master))
        return self
    
    def encode_jockey_id(self):
        """
        jockey_idをラベルエンコーディングして、Categorical型に変換する。
        """
        csv_path = 'data/master/jockey_id.csv'
        jockey_master = pd.read_csv(csv_path, dtype=object)
        new_jockeys = self.__data[[Cols.JOCKEY_ID]][
            ~self.__data[Cols.JOCKEY_ID].isin(jockey_master['jockey_id'])
            ].drop_duplicates(subset=['jockey_id'])
        new_jockeys['encoded_id'] = [i+len(jockey_master) for i in range(len(new_jockeys))]
        new_jockey_master = pd.concat([jockey_master, new_jockeys]).set_index('jockey_id')['encoded_id']
        new_jockey_master.to_csv(csv_path)
        self.__data[Cols.JOCKEY_ID] = pd.Categorical(self.__data[Cols.JOCKEY_ID].map(new_jockey_master))
        return self
    
    def dumminize_kaisai(self):
        self.__data[Cols.KAISAI] = pd.Categorical(
            self.__data[Cols.KAISAI], list(Master.PLACE_DICT.values())
            )
        self.__data = pd.get_dummies(self.__data, columns=[Cols.KAISAI])
        return self